In [1]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

In [2]:
# Load CSV data into a pandas DataFrame
df = pd.read_csv('data/Battery_RUL_cleaned.csv')
df = df[df.columns[1:]]  # Remove the first column

# Last column is the target variable
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Create a TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create data loaders
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define models
class NeuralNetwork(nn.Module):
    def __init__(self, h_size):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(7, h_size),
            nn.ReLU(),
            nn.Linear(h_size, h_size//2),
            nn.ReLU(),
            nn.Linear(h_size//2, h_size//4),
            nn.ReLU(),
            nn.Linear(h_size//4, 1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
class NN(nn.Module):
    def __init__(self, h_size, d_rate):
        super().__init__()
        self.Flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(7, h_size),
            nn.BatchNorm1d(h_size),
            nn.ReLU(),
            nn.Dropout(d_rate),
            nn.Linear(h_size, h_size//2),
            nn.BatchNorm1d(h_size//2),
            nn.ReLU(),
            nn.Linear(h_size//2, h_size//4),
            nn.BatchNorm1d(h_size//4),
            nn.ReLU(),
            nn.Linear(h_size//4, 1)
        )

    def forward(self, x):
        x = self.Flatten(x)
        logit = self.linear_relu_stack(x)
        return logit
    

    
    
    
model = NN(512, 0.2).to(device)
#model = NeuralNetwork(512).to(device)
print(model)

Using cuda device
NN(
  (Flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=7, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Linear(in_features=256, out_features=128, bias=True)
    (8): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Linear(in_features=128, out_features=1, bias=True)
  )
)


In [4]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [5]:
def train(dataloader, model, loss1, loss2, optimizer, epoch):
    model.train()
    for batch in dataloader:
        X, y = batch
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = 0.7 * loss1(pred.squeeze(), y) + 0.3 * loss2(pred.squeeze(), y)
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Log the loss
        writer.add_scalar('Training Loss', loss.item(), epoch)
        return loss


In [6]:
def test(dataloader, model, loss1, loss2, epoch):
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            X, y = batch
            X, y = X.to(device), y.to(device)
            pred = model(X).squeeze()
            test_loss += loss1(pred, y).item() + loss2(pred, y).item()
    test_loss /= num_batches

    # Log the loss
    writer.add_scalar('Test Loss', test_loss, epoch)
    return test_loss


In [7]:
# Initialize TensorBoard writer
writer = SummaryWriter('runs/experiment_4')

loss1 = nn.L1Loss()
loss2 = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.8)
epochs = 1500

# Initialize early stopping
early_stopping = EarlyStopping(patience=50, min_delta=0.01)

pbar = tqdm(range(epochs))
for t in pbar:
    train_loss = train(train_dataloader, model, loss1, loss2, optimizer, t)
    test_loss = test(test_dataloader, model, loss1, loss2, t)
    scheduler.step()
    pbar.set_description(f"Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")

    # Check for early stopping
    #early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping")
        break

print("Done!")
torch.save(model.state_dict(), 'models/model.pt')  # Save only the state_dict

# Close the TensorBoard writer
writer.close()

model.eval()
X, y = next(iter(test_dataloader))
X, y = X.to(device), y.to(device)
with torch.no_grad():
    pred = model(X)
    predicted, actual = pred[0], y[0]
    print(f'Input: "{X[0]}", Predicted: "{predicted}", Actual: "{actual}"')

Train Loss: 33.6796, Test Loss: 69.4819: 100%|██████████| 1500/1500 [02:05<00:00, 11.97it/s] 


Done!
Input: "tensor([1.8370e+03, 5.7400e+02, 3.9660e+00, 3.4860e+00, 3.8748e+03, 4.7964e+03,
        8.6528e+03], device='cuda:0')", Predicted: "tensor([773.5727], device='cuda:0')", Actual: "827.0"
